In [2]:
!pip install google-generativeai

In [6]:
import pandas as pd

file_path = "./prepro_company_data.csv"
df = pd.read_csv(file_path)
df["Positive_Score"] = None
df

,company,code,Date,Title,Positive_Score
0,삼성전자,'005930,2021-03-12,진짜 5 G 28 서비스 활성화 첫 발 과기부 TF 발족 회의,None
1,삼성전자,'005930,2021-03-05,아버님 댁 에어 프라이어 놓다 드리다,None
2,삼성전자,'005930,2021-05-14,ESG 경영 첫 단추 늘다 여성 관리자 키우다,None
3,삼성전자,'005930,2021-02-05,삼성 공장 증설 19조 투자 주 정부 9000억 세제 요청,None
4,삼성전자,'005930,2021-01-22,화웨이 서다 분리 중 가폰 어너 인텔 삼성 반도체 계약,None
...,...,...,...,...,...
15895,코오롱인더,'120110,2023-07-21,연구개발 1.1조 쏟다 코오롱 인 더 하반기 과실 딴다,None
15896,코오롱인더,'120110,2023-10-20,코오롱 인 더 3 Q 영업 익 시장 기 대치 회 4 Q 패션 성 수기 효과 기대 IBK,None
15897,코오롱인더,'120110,2023-10-27,코오롱 몰다 파격 할인 2023 OLO 모드 K 쇼핑 위 크다 진행,None
15898,코오롱인더,'120110,2023-12-01,추다 날씨 아우 터 판매 확대 코오롱 인 더 실적 개선 전망 IBK,None


In [2]:
import google.generativeai as genai
import os
import re
import time
import pandas as pd

file_path = "./Gemini_1.5_SI.csv"
df2 = pd.read_csv("./KR-Finbert.csv")

# 파일 존재 여부 확인 및 초기화
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df = pd.DataFrame(columns=['Title', 'company', 'Positive_Score'])
    df.to_csv(file_path, index=False, encoding='utf-8-sig')

# Google API 키 설정
GOOGLE_API_KEY = "your_key_here"
genai.configure(api_key=GOOGLE_API_KEY)

# 보안 설정
safety_settings = [
    {"category": "HARM_CATEGORY_DANGEROUS", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

# 모델 초기화 (시스템 명령 포함)
system_instruction = "금융전문가로 가정해주세요. 당신은 주식 추천 경험이 있는 금융 전문가입니다."

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    safety_settings=safety_settings,
    system_instruction=system_instruction
)

chat = model.start_chat(history=[])

# 감성 분석 및 점수 저장
for idx, row in df.iterrows():
    title = row['Title']
    retry_count = 0
    max_retries = 3
    company = row['company']
    positive_score_from_df2 = df2.loc[df2['Title'] == title, 'Positive_Score'].values[0]

    # 'Positive_Score' 값이 이미 있는 경우 건너뛰기
    if pd.notnull(row['Positive_Score']):
        continue

    while retry_count < max_retries:
        try:
            print(f"Processing headline {idx}: {title}")
            prompt = f"헤드라인: {title}. 좋은 뉴스, 나쁜 뉴스, 불확실한 뉴스 세 가지가 될 확률이 합쳐서 1일 때 좋은 뉴스일 정도를 확률로만 답변해주세요. (예시답변: 0.2) {company}에 좋은 영향을 숫자로만 답해주세요. KR-FinBERT의 'Positive_Score'는 {positive_score_from_df2}입니다."
            response = chat.send_message(prompt)

            # 응답에서 숫자만 추출하여 저장
            score = re.findall(r'\d+\.\d+', response.text)
            if score:
                df.at[idx, 'Positive_Score'] = float(score[0])

            # 결과 저장 (한글 인코딩 옵션 추가)
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            time.sleep(1)
            break
        
        except Exception as e:
            print(f"Error occurred: {e}. Retrying in 5 seconds...")
            retry_count += 1
            time.sleep(5)
            model = genai.GenerativeModel(
                model_name="gemini-1.5-pro-latest",
                safety_settings=safety_settings,
                system_instruction=system_instruction
            )
            chat = model.start_chat(history=[])
        
        if retry_count == max_retries:
            print(f"Maximum retry limit reached for index {idx}. Skipping...")
            continue  # 최대 재시도 횟수에 도달했다면 다음 루프로 넘어갑니다.
    
    df = pd.read_csv(file_path)

print("완료되었습니다.")


1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509


2320
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2320
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2320. Skipping...
2321
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2321
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2321
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2321. Skipping...
2322
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2322
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2322
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2322. Skipping...
2323
Error occurred: 429 Resource has be

2343
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2343. Skipping...
2344
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2344
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2344
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2344. Skipping...
2345
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2345
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2345
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2345. Skipping...
2346
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2346
Error occurred: 429 Resource has be

Maximum retry limit reached for index 2366. Skipping...
2367
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2367
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2367
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2367. Skipping...
2368
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2368
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2368
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2368. Skipping...
2369
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2369
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2369
Error occurred: 429 Resource has be

2390
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2390
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2390. Skipping...
2391
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2391
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2391
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2391. Skipping...
2392
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2392
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
2392
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 2392. Skipping...
2393
Error occurred: 429 Resource has be

...

3183
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3183. Skipping...
3184
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3184
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3184
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3184. Skipping...
3185
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3185
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3185
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3185. Skipping...
3186
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3186
Error occurred: 429 Resource has be

Maximum retry limit reached for index 3206. Skipping...
3207
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3207
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3207
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3207. Skipping...
3208
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3208
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3208
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3208. Skipping...
3209
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3209
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3209
Error occurred: 429 Resource has be

3230
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3230
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3230. Skipping...
3231
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3231
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3231
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3231. Skipping...
3232
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3232
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3232
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3232. Skipping...
3233
Error occurred: 429 Resource has be

3253
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3253. Skipping...
3254
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3254
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3254
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3254. Skipping...
3255
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3255
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3255
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
Maximum retry limit reached for index 3255. Skipping...
3256
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...
3256
Error occurred: 429 Resource has be

KeyboardInterrupt: 